In [3]:
from pillow_heif import register_heif_opener
from PIL import Image
import os
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

class HeicConverter:
    def __init__(self):
        # Регистрируем поддержку HEIC один раз при инициализации
        register_heif_opener()
        
    def convert_single_file(self, input_path: Path, output_path: Path) -> bool:
        """
        Конвертирует одиночный HEIC файл в PNG
        """
        try:
            with Image.open(input_path) as image:
                output_path.parent.mkdir(parents=True, exist_ok=True)
                image.save(output_path, format='PNG')
            return True
        except Exception as e:
            print(f'Ошибка при конвертации {input_path}: {str(e)}')
            return False

    def convert_directory(self, input_dir: str, output_dir: str, max_workers: int = 4):
        """
        Конвертирует все HEIC файлы из входной директории в выходную
        
        Args:
            input_dir (str): Путь к директории с HEIC файлами
            output_dir (str): Путь к директории для сохранения PNG файлов
            max_workers (int): Количество параллельных потоков
        """
        # Преобразуем пути в объекты Path
        input_path = Path(input_dir)
        output_path = Path(output_dir)
        
        # Получаем список всех HEIC файлов
        heic_files = list(input_path.glob('**/*.HEIC')) + list(input_path.glob('**/*.heic'))
        
        if not heic_files:
            print("HEIC файлы не найдены в указанной директории")
            return
        
        print(f"Найдено {len(heic_files)} HEIC файлов")
        
        # Создаем список задач для конвертации
        conversion_tasks = []
        for input_file in heic_files:
            # Сохраняем структуру папок при конвертации
            relative_path = input_file.relative_to(input_path)
            output_file = output_path / relative_path.with_suffix('.png')
            conversion_tasks.append((input_file, output_file))
        
        # Выполняем конвертацию в параллельных потоках
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            results = list(tqdm(
                executor.map(lambda x: self.convert_single_file(*x), conversion_tasks),
                total=len(conversion_tasks),
                desc="Конвертация файлов"
            ))
        
        # Выводим статистику
        successful = sum(results)
        print(f"\nКонвертация завершена:")
        print(f"Успешно: {successful}")
        print(f"С ошибками: {len(results) - successful}")

def main():
    # Пример использования
    converter = HeicConverter()
    
    input_directory = "/Users/nikitaanurov/Downloads/Командировка в Ухту"  # Укажите путь к папке с HEIC файлами
    output_directory = "/Users/nikitaanurov/Downloads/Sosnogorsk_Nikita_iPhone"  # Укажите путь куда сохранять PNG файлы
    
    converter.convert_directory(input_directory, output_directory)

if __name__ == "__main__":
    main()

Найдено 348 HEIC файлов


Конвертация файлов: 100%|██████████| 348/348 [20:04<00:00,  3.46s/it]


Конвертация завершена:
Успешно: 348
С ошибками: 0
